In [1]:
import importlib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
from analysis_functions import *

In [3]:
importlib.reload(sys.modules['analysis_functions'])
from analysis_functions import *

In [4]:
X_stress_components_new = opener('X_stress_components_new', path_import='./resourses/')
X_strain_components_new = opener('X_strain_components_new', path_import='./resourses/')
y_stress_components_new = opener('y_stress_components_new', path_import='./resourses/')
y_strain_components_new = opener('y_strain_components_new', path_import='./resourses/')
# X_stress_components_other = opener('X_stress_components_other')
# X_strain_components_other = opener('X_strain_components_other')
# y_stress_components_other = opener('y_stress_components_other')
# y_strain_components_other = opener('y_strain_components_other')

./resourses//X_stress_components_new.pkl
./resourses//X_strain_components_new.pkl
./resourses//y_stress_components_new.pkl
./resourses//y_strain_components_new.pkl


In [5]:
X_stress_components_new.shape

(3, 2678, 5)

In [6]:
component_num = 1
n_trials = 200

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')

In [8]:
import shutil
import numpy as np
from sklearn.metrics import mean_squared_error

def plot_results(x, y, model_name, do_zip=False, dpi=500, delete_and_rewrite=True):
    cur_X_test, cur_y_test, *_ = get_train_test(x, y)

    model = opener(model_name)
    cur_prediction = model.predict(cur_X_test)
    print('rmse = ', np.sqrt(mean_squared_error(cur_y_test, cur_prediction)))
    
    if delete_and_rewrite:
        os.makedirs(model_name, exist_ok=True)
    
    for i, item in enumerate(cur_X_test[::20]):
        fig = plt.figure(figsize=(5, 4))
        plt.plot(
            cur_X_test[i*20:(i+1)*20, -1],
            cur_prediction[i*20:(i+1)*20],
            '-o',
            label='predicted'
        )
        plt.plot(
            cur_X_test[i*20:(i+1)*20, -1],
            cur_y_test[i*20:(i+1)*20],
            '-o',
            label='real'
        )
        title = f'red {item[0]}, cal {item[1]}, alpha {int(item[2])}, fric {item[3]}'
        plt.title(title)
        plt.legend(loc='best')
        plt.xlabel('r, -')
        plt.ylabel('$\sigma$, MPa')
        # plt.show()
        plt.savefig(''.join((model_name, '/', title, '.png')), format='png', dpi=dpi)
        plt.close()
        # break
    if do_zip: shutil.make_archive(model_name, 'zip', model_name)

In [9]:
X_stress_components_new[0].max(axis=0)

array([ 0.25,  1.  , 20.  ,  0.1 , 40.  ])

# KAN 

In [10]:
import importlib
import sys
from kan import KAN
KANModelTrainTest = importlib.reload(sys.modules["analysis_functions"]).KANModelTrainTest

In [11]:
kan_model:KANModelTrainTest = KANModelTrainTest() # type: ignore

In [ ]:
kan_model.create_train_val_test(
    X=X_stress_components_new[0], y=y_stress_components_new[0], n_splits=1
)


In [18]:

kan_model.create_dataset(
    kan_model.train_set_X[0],
    kan_model.train_set_y[0],
    kan_model.val_set_X[0],
    kan_model.val_set_y[0],
)['train_input'].shape


torch.Size([37480, 6])

In [14]:
orig_kan = KAN([6, 3, 2, 1], seed=RANDOM_STATE)

orig_kan.fit(
    kan_model.create_dataset(
        kan_model.train_set_X[0],
        kan_model.train_set_y[0],
        kan_model.val_set_X[0],
        kan_model.val_set_y[0],
    ),
)

checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/100 [01:40<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
kan_model.train_model(
    width=[6, 3, 2, 1],
    opt='LBFGS',
    steps=5
)

checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                            | 0/5 [00:00<?, ?it/s]

In [13]:
kan_model.optimize_hyperparams(n_trials=100, max_n_layers=4, max_n_units=5, max_steps=300)

[I 2025-03-26 23:08:46,397] A new study created in memory with name: no-name-53b1214f-c22d-4dde-a2fb-3c42909f84c8
2025-03-26 23:08:46,402 - INFO - width = [6, 5, 2, 1]


checkpoint directory created: ./model
saving model version 0.0


| train_loss: 3.90e+01 | test_loss: 3.91e+01 | reg: 0.00e+00 | :   1%| | 1/139 [1:17:53<179:08:32, 4
[W 2025-03-27 00:26:39,812] Trial 0 failed with parameters: {'n_layers': 2, 'opt': 'LBFGS', 'steps': 139, 'n_units_0': 5, 'n_units_1': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/analysis_functions.py", line 1302, in objective
    list_val_rmse = self.train_model(width=width, opt=opt, steps=steps)
  File "/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/analysis_functions.py", line 1277, in train_model
    result: dict = kan_model.fit(my_dataset, opt=opt, steps=steps)
  File "/Users/i.grebenkin/pythonProjects/учеба/wire_drawing/.venv/lib/python3.9/site-packages/kan/MultKAN.py", line 1552, in fit
   

KeyboardInterrupt: 

In [ ]:
(kan_model.input_layer, kan_model.output_layer,)


In [ ]:
len(kan_model.output_layer.shape)